A notebook for looking at the results of one run of syn_ahrens_gnlr_mdls

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import pathlib
import pickle

import matplotlib.pyplot as plt
import numpy as np
import torch

from ahrens_wbo.data_processing import generate_torch_dataset
from ahrens_wbo.data_processing import load_and_preprocess_data
from ahrens_wbo.data_processing import SegmentTable
from janelia_core.stats.regression import r_squared
from probabilistic_model_synthesis.gaussian_nonlinear_regression import predict
from probabilistic_model_synthesis.gaussian_nonlinear_regression import Fitter
from probabilistic_model_synthesis.gaussian_nonlinear_regression import PriorCollection
from probabilistic_model_synthesis.gaussian_nonlinear_regression import VICollection

In [3]:
%matplotlib notebook

## Parameters go here

In [4]:
ps = dict()

# Specify the location of results
ps['results_dir'] = r'/groups/bishop/bishoplab/projects/probabilistic_model_synthesis/results/real_data/gnlr/same_cond_transfer_analysis/v18/fold_str_base_14_tgt_14/fold_1/subj_11/comb'
ps['results_file'] = r'test_results.pt'

# Specify if we want to look at sp or ip fits
ps['mdl_type'] = 'ip'

# Specify the subjects we use for validation across check points
ps['validation_subjects'] = [11]

# Specify if we use retrospective early stopping based on validation performance across check points
ps['use_early_stopping'] = True



## Load the results

In [5]:
rs_path = pathlib.Path(ps['results_dir']) / ps['results_file']
rs = torch.load(rs_path)

In [6]:
fit_rs = rs['rs'][ps['mdl_type']] 
fit_ps = rs['ps']
subject_order = rs['subject_order']
fold = fit_ps['fold']

In [7]:
fit_ps

{'note': 'Initial testing.',
 'param_filename': 'transfer_params.pkl',
 'param_save_dir': '/groups/bishop/bishoplab/projects/probabilistic_model_synthesis/results/real_data/gnlr/same_cond_transfer_analysis/v18',
 'results_dir': '/groups/bishop/bishoplab/projects/probabilistic_model_synthesis/results/real_data/gnlr/same_cond_transfer_analysis/v18/fold_str_base_14_tgt_14/fold_1/subj_11/comb',
 'save_file': 'test_results.pt',
 'sp_cp_dir': '/groups/bishop/bishoplab/projects/probabilistic_model_synthesis/results/real_data/gnlr/same_cond_transfer_analysis/v18/fold_str_base_14_tgt_14/fold_1/subj_11/comb/sp_cp',
 'ip_cp_dir': '/groups/bishop/bishoplab/projects/probabilistic_model_synthesis/results/real_data/gnlr/same_cond_transfer_analysis/v18/fold_str_base_14_tgt_14/fold_1/subj_11/comb/ip_cp',
 'data_dir': '/groups/bishop/bishoplab/projects/ahrens_wbo/data',
 'segment_table_dir': '/groups/bishop/bishoplab/projects/probabilistic_model_synthesis/results/real_data',
 'segment_table_file': 'phot

## View the fitting logs

In [ ]:
for log in fit_rs['logs']:
    Fitter.plot_log(log)

## Load segment tables, fold structures and data

Load segment tables

In [ ]:
segment_table_path = pathlib.Path(fit_ps['segment_table_dir']) / fit_ps['segment_table_file']
with open(segment_table_path, 'rb') as f:
    st_data = pickle.load(f)
    segment_tables = st_data['segment_tables']
segment_tables = {k: SegmentTable.from_dict(v) for k, v in segment_tables.items()}

Load fold structures

In [ ]:
fold_str_path = pathlib.Path(fit_ps['fold_str_dir']) / fit_ps['fold_str_file']
with open(fold_str_path, 'rb') as f:
    fold_groups = pickle.load(f)

Load the data

In [ ]:
subject_data, subject_neuron_locs = load_and_preprocess_data(data_folder=fit_ps['data_dir'], 
                                            subjects=subject_order,
                                            normalize_beh_vars=fit_ps['normalize_beh_vars'],
                                            neural_gain=fit_ps['neural_gain'], 
                                            beh_gain=fit_ps['beh_gain'],
                                            z_ratio=fit_ps['z_ratio'])


Form the train, validation and test data 

In [ ]:
data = dict()
for s_n in subject_order:
    subject_train_slices = segment_tables[s_n].find_all(fold_groups[s_n][fold]['train'])
    train_data_n = generate_torch_dataset(dataset=subject_data[s_n], 
                                          slices=subject_train_slices,
                                          ts_keys=['dff', 'behavior'], inc_time_stamps=True)
    
    train_data_n = (train_data_n.data[0][train_data_n.i_x, :], 
                    train_data_n.data[1][train_data_n.i_y, :],
                    train_data_n.data[2][train_data_n.i_y])

    
    
    subject_test_slices = segment_tables[s_n].find_all(fold_groups[s_n][fold]['test'])
    test_data_n = generate_torch_dataset(dataset=subject_data[s_n], 
                                         slices=subject_test_slices,
                                         ts_keys=['dff', 'behavior'], inc_time_stamps=True)
    
    test_data_n = (test_data_n.data[0][test_data_n.i_x, :], 
                   test_data_n.data[1][test_data_n.i_y, :],
                   test_data_n.data[2][test_data_n.i_y])
    
    subject_validation_slices = segment_tables[s_n].find_all(fold_groups[s_n][fold]['validation'])
    validation_data_n = generate_torch_dataset(dataset=subject_data[s_n], 
                                               slices=subject_validation_slices,
                                               ts_keys=['dff', 'behavior'], 
                                               inc_time_stamps=True)
    
    validation_data_n = (validation_data_n.data[0][validation_data_n.i_x, :], 
                         validation_data_n.data[1][validation_data_n.i_y, :],
                         validation_data_n.data[2][validation_data_n.i_y])
    
    data[s_n] = {'train': train_data_n, 
                 'test': test_data_n, 
                 'validation': validation_data_n}

## Perform retrospective early stopping to pick the best model

Load checkpoints

In [ ]:
cp_dir = pathlib.Path(fit_ps[ps['mdl_type'] + '_cp_dir'])
cp_files = glob.glob(str(cp_dir / 'cp_*.pt'))
n_cps = len(cp_files)
cp_rs = [None]*n_cps
for cp_i, cp_file in enumerate(cp_files):
    cp_rs[cp_i] = torch.load(cp_file)
    print('Done loading check point ' + str(cp_i + 1) + ' of ' + str(n_cps) + '.')

Evaluate performance at each check point

In [ ]:
validation_subject_inds = [np.argwhere(np.asarray(subject_order) == s_n)[0][0] 
                               for s_n in ps['validation_subjects']]
n_validation_stopping_subjects = len(validation_subject_inds)

In [ ]:
cp_performance = np.zeros([n_cps, n_validation_stopping_subjects])
for cp_i, cp in enumerate(cp_rs):
    for i, (s_i, s_n) in enumerate(zip(validation_subject_inds, ps['validation_subjects'])):
        x = data[s_n]['validation'][0]
        y = data[s_n]['validation'][1].numpy()
        coll = VICollection.from_checkpoint(cp['vi_collections'][s_i], props=subject_neuron_locs[s_n])
        y_hat = predict(coll, x).detach().numpy()
        cp_performance[cp_i, i]  = np.mean(r_squared(truth=y, pred=y_hat))

Sort checkpoints by epoch

In [ ]:
cp_epochs = np.asarray([cp['total_epoch'] for cp in cp_rs])
cp_sort_order = np.argsort(cp_epochs)
cp_epochs = cp_epochs[cp_sort_order]
cp_rs = [cp_rs[i] for i in cp_sort_order]
cp_performance = cp_performance[cp_sort_order]

View performance across check points

In [ ]:
mean_cp_performance = np.mean(cp_performance, axis=1)

plt.figure()
plt.plot(cp_epochs, cp_performance)
plt.plot(cp_epochs, mean_cp_performance, 'k--')

Pick the best check point here

In [ ]:
if ps['use_early_stopping']:
    best_cp_ind = np.argmax(mean_cp_performance)
    best_cp = cp_rs[best_cp_ind]
    
    best_vi_collections = [VICollection.from_checkpoint(best_cp['vi_collections'][s_i], 
                                                        props=subject_neuron_locs[s_n])
                           for s_i, s_n in enumerate(subject_order)]
    
    best_priors = PriorCollection.from_checkpoint(best_cp['priors'])
    
    print('Best CP epoch: ' + str(cp_epochs[best_cp_ind]))
else:
    best_vi_collections = fit_rs['vi_collections']
    best_priors = fit_rs['priors']

## View predictions for the best performing models

In [ ]:
ex_s_n = ps['validation_subjects'][0]
vis_data = 'validation'

In [ ]:
ex_s_i = np.argwhere(np.asarray(subject_order) == ex_s_n)[0][0]

In [ ]:
t = data[subject_order[ex_s_i]][vis_data][2].numpy()
x = data[subject_order[ex_s_i]][vis_data][0]
y = data[subject_order[ex_s_i]][vis_data][1]
y_hat = predict(best_vi_collections[ex_s_i], x).detach()

In [ ]:
r_squared(y.numpy(), y_hat.numpy())

In [ ]:
plt.figure()

plt.subplot(2,1,1)
plt.plot(t, y[:,0])
plt.plot(t, y_hat[:,0])
plt.legend(['truth', 'pred'])

plt.subplot(2,1,2)
plt.plot(t, y[:,1])
plt.plot(t, y_hat[:,1])
plt.legend(['truth', 'pred'])


## Debug code goes here

In [ ]:
for s_i, s_n in enumerate(subject_order):
    print('Subject: ' + str(s_n))
    
    s_in_mn = best_vi_collections[s_i].posteriors.s_in_post(torch.zeros(1)).detach().numpy().squeeze()
    s_in_std = best_vi_collections[s_i].posteriors.s_in_post.dists[0].std_f(torch.zeros(1)).detach().numpy().squeeze()
    print('S_in Mean: ' + str(s_in_mn))
    print('S_in Std: ' + str(s_in_std))
    
    b_in_mn = best_vi_collections[s_i].posteriors.b_in_post(torch.zeros(1)).detach().numpy().squeeze()
    b_in_std = best_vi_collections[s_i].posteriors.b_in_post.dists[0].std_f(torch.zeros(1)).detach().numpy().squeeze()
    print('B_in Mean: ' + str(b_in_mn))
    print('B_in Std: ' + str(b_in_std))
    
    s_out_mn = best_vi_collections[s_i].posteriors.s_out_post(torch.zeros(1)).detach().numpy().squeeze()
    s_out_std = best_vi_collections[s_i].posteriors.s_out_post.dists[0].std_f(torch.zeros(1)).detach().numpy().squeeze()
    print('S_out Mean: ' + str(s_out_mn))
    print('S_out Std: ' + str(s_out_std))
    
    
    
    b_out_mn = best_vi_collections[s_i].posteriors.b_out_post(torch.zeros(1)).detach().numpy().squeeze()
    b_out_std = best_vi_collections[s_i].posteriors.b_out_post.dists[0].std_f(torch.zeros(1)).detach().numpy().squeeze()
    print('B_out Mean: ' + str(b_out_mn))
    print('B_out Std: ' + str(b_out_std))

In [ ]:
best_priors.s_out_prior.dists[0].mn_f(torch.zeros(1))